<a href="https://colab.research.google.com/github/singwang-cn/The_first_pattern_recognition/blob/master/ex8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn import svm

In [176]:
class support_vector_machine():
  def __init__(self, kernel='rbf', sigma=4, c=10):
    self.kernel = kernel
    self.sigma = sigma
    self.c = c
    self.w = None
    self.b = None
    self.l = None
    self.t = None
    self.H = None
    self.alpha = None
    self.cons = (
      {'type': 'eq', 'fun': lambda x: np.dot(x, self.t)}
    )
  def obj_func(self, alpha):
    return 1/2*np.dot(np.dot(alpha, self.H), alpha)-np.sum(alpha)
  def fit(self, X, Y):
    n, d = X.shape
    self.l = X
    self.t = np.where(Y==0, -1, 1)
    self.H = np.zeros((n, n))
    for i in range(n):
      for j in range(n):
        self.H[i][j] = self.t[i]*self.t[j]*self.rbf_kernel(X[i], X[j])
    self.alpha = minimize(self.obj_func, 
                        x0=np.random.rand(n), bounds=((0, self.c),)*n,
                        constraints=self.cons, method="SLSQP").x
  def predict(self, x):
    n = self.alpha.shape[0]
    result = np.zeros(n)
    for i in range(n):
      result[i] = self.alpha[i]*self.t[i]*self.rbf_kernel(self.l[i], x)
    if np.sum(result) > 0:
      return 1
    else:
      return 0
  def rbf_kernel(self, u, v):
    norm = np.linalg.norm(u-v, ord=2)
    return np.exp(-1/(2*self.sigma)*(norm**2))
    

In [110]:
data_path = '/content/drive/My Drive/Dataset/diabetes.csv'
pima = pd.read_csv(data_path)
pima_label = np.array(pima['Outcome'].tolist())
pima_nolabel = pima.drop(columns=['Outcome'])
pima_data = ((pima_nolabel-pima_nolabel.mean())/pima_nolabel.std()).values

In [162]:
svm = support_vector_machine()
svm.fit(pima_data, pima_label)

1.3025513081686242

In [174]:
correct = 0
for i in range(pima_label.shape[0]):
  if svm.predict(pima_data[i])>0 and pima_label[i]==1 or svm.predict(pima_data[i])<0 and pima_label[i]==0:
    print(svm.predict(pima_data[i]),',',pima_label[i])
    correct += 1
print(correct/pima_label.shape[0])

1.143227689635765 , 1
-1.2196731899120365 , 0
1.0963661180464817 , 1
-1.3719893193094137 , 0
1.0963213869674169 , 1
-1.129320503937148 , 0
-0.9036541443192565 , 0
2.6208925156079905 , 1
1.0963206494362865 , 1
-1.431929888728007 , 0
1.1265499334706908 , 1
-0.9037285548133462 , 0
1.0962289402198655 , 1
1.0962346354289423 , 1
0.5864334062061971 , 1
-0.9037255090858736 , 0
-0.32147493825785034 , 0
-1.452345914707017 , 0
2.532949624166569 , 1
0.04044788393923682 , 1
1.0028957395368294 , 1
1.0962707225677306 , 1
-2.0332928574812286 , 0
-0.9037005821838227 , 0
-1.1668764542156005 , 0
-0.49572738039652364 , 0
0.9921550112517963 , 1
-1.7955858133933944 , 0
-2.1683882674567947 , 0
-1.0173939073742702 , 0
0.3594452705516673 , 1
1.0963339252185218 , 1
-1.945416180336263 , 0
2.4977752930070523 , 1
1.0962744505903252 , 1
-1.5132419710115954 , 0
-0.9037480921786232 , 0
-2.177338702248697 , 0
-2.150826349670396 , 0
-1.484853321710311 , 0
2.2447771359236546 , 1
-2.148633811175067 , 0
1.9010260282090385

In [166]:
pima_label[1]

0